# Natural Language Database AI Assistant

- Take existing csv file as an example database table.
- Use pandas library to set csv file as temporary in-memory database table
- Test a manually created SQL command against the table.
- Use a coding LLM to translate natural language to SQL commands.
- Combine the steps together in a function call to go straight from natural language to results

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from vertexai.language_models import CodeGenerationModel

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ADD CREDENTIALS HERE'

In [3]:
df = pd.read_csv('penguins.csv')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


## Creating Temporary Database

In [4]:
temp_df = create_engine('sqlite:///:memory:', echo=True)

In [5]:
data = df.to_sql(name='Penguins', con=temp_df)

2023-11-27 18:52:56,673 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-27 18:52:56,685 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Penguins")
2023-11-27 18:52:56,686 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-27 18:52:56,688 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Penguins")
2023-11-27 18:52:56,689 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-27 18:52:56,692 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Penguins" (
	"index" BIGINT, 
	species TEXT, 
	island TEXT, 
	bill_length_mm FLOAT, 
	bill_depth_mm FLOAT, 
	flipper_length_mm FLOAT, 
	body_mass_g FLOAT, 
	sex TEXT
)


2023-11-27 18:52:56,693 INFO sqlalchemy.engine.Engine [no key 0.00090s] ()
2023-11-27 18:52:56,696 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Penguins_index" ON "Penguins" ("index")
2023-11-27 18:52:56,698 INFO sqlalchemy.engine.Engine [no key 0.00179s] ()
2023-11-27 18:52:56,706 INFO sqlalchemy.engine.Engine INSERT INTO "Penguins" ("index", species, island, bill_length_m

In [6]:
with temp_df.connect() as conn:
    result = conn.execute(text("SELECT * FROM Penguins"))

2023-11-27 18:52:56,728 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-27 18:52:56,730 INFO sqlalchemy.engine.Engine SELECT * FROM Penguins
2023-11-27 18:52:56,731 INFO sqlalchemy.engine.Engine [generated in 0.00305s] ()
2023-11-27 18:52:56,733 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
result.all()

[(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
 (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE'),
 (2, 'Adelie', 'Torgersen', 40.3, 18.0, 195.0, 3250.0, 'FEMALE'),
 (3, 'Adelie', 'Torgersen', None, None, None, None, None),
 (4, 'Adelie', 'Torgersen', 36.7, 19.3, 193.0, 3450.0, 'FEMALE'),
 (5, 'Adelie', 'Torgersen', 39.3, 20.6, 190.0, 3650.0, 'MALE'),
 (6, 'Adelie', 'Torgersen', 38.9, 17.8, 181.0, 3625.0, 'FEMALE'),
 (7, 'Adelie', 'Torgersen', 39.2, 19.6, 195.0, 4675.0, 'MALE'),
 (8, 'Adelie', 'Torgersen', 34.1, 18.1, 193.0, 3475.0, None),
 (9, 'Adelie', 'Torgersen', 42.0, 20.2, 190.0, 4250.0, None),
 (10, 'Adelie', 'Torgersen', 37.8, 17.1, 186.0, 3300.0, None),
 (11, 'Adelie', 'Torgersen', 37.8, 17.3, 180.0, 3700.0, None),
 (12, 'Adelie', 'Torgersen', 41.1, 17.6, 182.0, 3200.0, 'FEMALE'),
 (13, 'Adelie', 'Torgersen', 38.6, 21.2, 191.0, 3800.0, 'MALE'),
 (14, 'Adelie', 'Torgersen', 34.6, 21.1, 198.0, 4400.0, 'MALE'),
 (15, 'Adelie', 'Torgersen', 36.6, 17.

## LLMs to query Database

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [9]:
def create_prefix(query):
    prefix = f""" Return a SQL statement that answers the following query:
    {query}
    
    For a table called 'Penguins' with the following properties:
    
     #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object 

     Example Rows:
     (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
     (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE'),
     (2, 'Adelie', 'Torgersen', 40.3, 18.0, 195.0, 3250.0, 'FEMALE')

     Only return the SQL statement for the query
    """

    return prefix

In [10]:
def user_input():
    query = input("Ask a question about the Penguins table")
    return create_prefix(query)

In [11]:
prefix = user_input()

Ask a question about the Penguins table How many male penguins are there?


In [12]:
print(prefix)

 Return a SQL statement that answers the following query:
    How many male penguins are there?
    
    For a table called 'Penguins' with the following properties:
    
     #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object 

     Example Rows:
     (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
     (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE'),
     (2, 'Adelie', 'Torgersen', 40.3, 18.0, 195.0, 3250.0, 'FEMALE')

     Only return the SQL statement for the query
    


In [13]:
code_generation_model = CodeGenerationModel.from_pretrained('code-bison')

In [14]:
print("Hello, I am your AI database assistant")
print('\n\n')

prefix = user_input()
result = code_generation_model.predict(prefix=prefix)

Hello, I am your AI database assistant





Ask a question about the Penguins table How many male penguins are there?


In [15]:
print(result.text)

 SELECT COUNT(*)
FROM Penguins
WHERE sex = 'MALE';


In [16]:
def clean_sql(sql_statement):
    return sql_statement.replace('```sql','').replace('```','').replace('\n',' ')

In [17]:
clean_sql(result.text)

" SELECT COUNT(*) FROM Penguins WHERE sex = 'MALE';"

## Creating Wrapper Function

In [18]:
def nlp_assistant():
    print("Hello, I am your AI database assistant")
    print('\n\n')
    
    prefix = user_input()
    result = code_generation_model.predict(prefix=prefix)
    sql = clean_sql(result.text)

    with temp_df.connect() as conn:
        print("\n Ok I am running this SQL statement")
        result = conn.execute(text(sql))
    print("\n==========================================")
    print("The results are:")
    print(result.all())

In [19]:
nlp_assistant()

Hello, I am your AI database assistant





Ask a question about the Penguins table How many male penguins are there?



 Ok I am running this SQL statement
2023-11-27 18:53:34,411 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-27 18:53:34,413 INFO sqlalchemy.engine.Engine  SELECT COUNT(*) FROM Penguins WHERE sex = 'MALE';
2023-11-27 18:53:34,413 INFO sqlalchemy.engine.Engine [generated in 0.00308s] ()
2023-11-27 18:53:34,415 INFO sqlalchemy.engine.Engine ROLLBACK

The results are:
[(168,)]


In [20]:
nlp_assistant()

Hello, I am your AI database assistant





Ask a question about the Penguins table How many distinct island names are there?



 Ok I am running this SQL statement
2023-11-27 18:54:01,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-27 18:54:01,329 INFO sqlalchemy.engine.Engine  SELECT COUNT(DISTINCT island) FROM Penguins; 
2023-11-27 18:54:01,330 INFO sqlalchemy.engine.Engine [generated in 0.00182s] ()
2023-11-27 18:54:01,332 INFO sqlalchemy.engine.Engine ROLLBACK

The results are:
[(3,)]


In [21]:
nlp_assistant()

Hello, I am your AI database assistant





Ask a question about the Penguins table What are the unique island names?



 Ok I am running this SQL statement
2023-11-27 18:54:42,911 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-27 18:54:42,912 INFO sqlalchemy.engine.Engine  SELECT DISTINCT island FROM Penguins; 
2023-11-27 18:54:42,913 INFO sqlalchemy.engine.Engine [generated in 0.00159s] ()
2023-11-27 18:54:42,914 INFO sqlalchemy.engine.Engine ROLLBACK

The results are:
[('Torgersen',), ('Biscoe',), ('Dream',)]
